In [1]:
import keras
from keras import callbacks
from keras.datasets import mnist
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
RN_input_small = np.load('RN_input_small.npy')
a_small = np.load('a_small.npy')
print (RN_input_small.shape)
print (a_small.shape)

(1000, 190, 1536)
(1000,)


In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(a_small)
train_labels = le.transform(a_small)

In [4]:
np.unique(train_labels)
l = len(np.unique(train_labels))

In [5]:
c_sent_max_len = 12
q_sent_max_len = 12
embedding_size = 32
LSTM_size = 256
batch_size = 1

In [6]:
def convert_to_RN_input(embedded_c, embedded_q, answer):
    """
    Returns the input for RN which has dim (n, 190, 1536), 
    where n is the number of context and 190=20C2.
    Input(s):
    - embedded_c: list of context (up to 20 sentences) 
                  for each different question.  
    - embedded_q: embedding for the related question to 
                  the given embedded_c.
    - answer: corresponding answer to the embedded c and q.              
    """
    object_pairs = []

    for i in embedded_c:
        object_pair = list(itertools.combinations(i, 2))
        object_pairs.append(object_pair)
    
    rn_inputs = []
    m = len(object_pairs)
    z = np.zeros((m, 190, 1536))    
    
    for i, object_pair in enumerate(object_pairs):
        n = len(object_pair)
        for j in range(n):
            l = np.array(object_pair[j][0] + object_pair[j][1] + embedded_q[i][0])
#             z[i][j] = np.array(object_pair[j][0] + object_pair[j][1] + embedded_q[i][0])
            z[i][j] = np.concatenate((object_pair[j][0], object_pair[j][1], embedded_q[i][0]))
    return z

In [8]:
c_input = Input(shape=(c_sent_max_len,))
c_embed = Embedding(c_sent_max_len, embedding_size)(c_input)
c_lstm = LSTM(LSTM_size)(c_embed)

q_input = Input(shape=(q_sent_max_len,))
q_embed = Embedding(q_sent_max_len, embedding_size)(q_input)
q_lstm = LSTM(LSTM_size)(q_embed)

# RN_inputs = Lambda(convert_to_RN_input, arguments={c_lstm,q_lstm})
RN_inputs = concatenate([c_lstm, q_lstm])
RN_inputs = Reshape((512,1))(RN_inputs)

factor = 1

g_units = [256,256,256,256]*factor

g1 = Dense(g_units[0], activation='relu')(RN_inputs)
g2 = Dense(g_units[1], activation='relu')(g1)
g3 = Dense(g_units[2], activation='relu')(g2)
g4 = Dense(g_units[3], activation='relu')(g3)
g_sum = Lambda(lambda x : K.sum(x,axis=1))(g4)

f_units = [256*factor, 512*factor, l]

f1 = Dense(f_units[0], activation='relu')(g_sum)
f2 = Dense(f_units[1], activation='relu')(f1)
f3 = Dense(f_units[2], activation='softmax')(f2)


In [9]:
model = Model(inputs=[c_input,q_input], outputs = f3)

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 12)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 12, 32)       384         input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 12, 32)       384         input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS